# Enrichment Analysis Visualizer

This appyter creates a variety of visualizations for enrichment analysis results for one selected Enrichr library, and may be run as either a standalone appyter from the [Appyter Catalog](https://appyters.maayanlab.cloud/#/Enrichment_Analysis_Visualizer) or programmatically from the [Enrichr](https://maayanlab.cloud/Enrichr/) results page. 

For simplicity, the only inputs for this appyter are a gene list and one library. Other parameters are set to default values in the cell below. You can download the notebook, change these parameters, and rerun it if you wish.

The pre-processed libraries used to create the scatter plot and hexagonal canvas visualizations can be found [here](https://github.com/MaayanLab/Enrichr-Viz-Appyter/tree/master/Enrichr-Processed-Library-Storage). 

**A link to the full analysis results on the Enrichr website can be found at the bottom of this page.**

In [2]:
# Scatter Plot Imports
from maayanlab_bioinformatics.enrichment import enrich_crisp
import matplotlib as mpl

# Bar Chart Imports
import pandas as pd 
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import time
from matplotlib.ticker import MaxNLocator
from IPython.display import display, FileLink, HTML, Markdown

# Hexagonal Canvas Imports
import math
import uuid
import urllib
from textwrap import dedent
from string import Template
from operator import itemgetter

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [3]:
gene_list_input = ['SMARCA1', 'FRS2', 'ATG14', 'ATG13', 'SRRM2', 'PML', 'ZFYVE16', 'KATNAL2', 'TEK', 'ASF1A', 'ULK1', 'CDK13', 'ASB6', 'RB1', 'PRKAA2', 'CCNK', 'SATB2', 'ARPC5L', 'PRKAA1', 'CRKL', 'DPYSL2', 'RBM15B', 'DPYSL5', 'CHEK1', 'BSG', 'DPYSL3', 'DPYSL4', 'AKT1', 'TIPIN', 'INSR', 'PRKCB', 'PRKCA', 'BAZ1B', 'ARFGAP2', 'PBX1', 'PTK2', 'BTC', 'TPX2', 'PIK3R1', 'GRB2', 'HOXB9', 'C9ORF72', 'FOXE1', 'PRKDC', 'ELK1', 'ICAM1', 'LARP1', 'SOX2', 'SNRNP70', 'MLST8', 'SYNPO', 'APOB', 'LYN', 'EDN1', 'VEGFC', 'PRPF40A', 'TUBB2A', 'NCOR1', 'CEACAM1', 'MAPT', 'KLHL9', 'ACTR10', 'LIMS1', 'POT1', 'ARPC1A', 'DDR1', 'MTTP', 'SDC4', 'PEA15', 'TGFA', 'ADRB2', 'FXR2', 'BCLAF1', 'GAR1', 'TMED3', 'BRD4', 'SVIL', 'CSNK2A2', 'CSNK2A1', 'RPA2', 'RPA3', 'RRP1B', 'RPA1', 'HCK', 'NFIB', 'CTTN', 'NFIC', 'C21ORF2', 'TIMELESS', 'HBEGF', 'CD44', 'FOXA3', 'FOXA1', 'DPYS', 'PRKAG3', 'RSF1', 'ASH2L', 'TCTN3', 'ENPP1', 'RIOK1', 'AP3S2', 'RICTOR', 'PTPN1', 'SEC24A', 'CAPZA2', 'PPIG', 'ROR1', 'PTPN6', 'CD81', 'AURKA', 'PRKX', 'EGFR', 'ARHGAP44', 'AP3M2', 'RBBP5', 'SH3BP1', 'AP3M1', 'TRAPPC2L', 'TRAPPC3', 'TRAPPC4', 'TRAPPC1', 'NEK5', 'TRAPPC8', 'PRNP', 'HGF', 'TNFRSF10A', 'EEF2', 'REEP5', 'EIF4G1', 'CSF1R', 'PXN', 'CUL5', 'KMT2D', 'RIF1', 'KLHL13', 'RLF', 'DPY30', 'ARHGAP17', 'ETS1', 'IGF1R', 'CHAF1A', 'EEF2K', 'HELB', 'CLSPN', 'SNIP1', 'NEK1', 'EIF4EBP2', 'ZC3H18', 'ABL1', 'CBX5', 'ITGA4', 'CBX3', 'SMARCE1', 'PARVB', 'HMGA1', 'TICRR', 'MERTK', 'THRAP3', 'SMCR8', 'PLEC', 'EIF4E2', 'BCAR1', 'RTN3', 'KDM5C', 'EIF4B', 'GSK3A', 'UBA5', 'CHD8', 'CHD7', 'TARS', 'FOXK1', 'GHR', 'PAGR1', 'DNAJB2', 'P2RY6', 'ERBB2', 'GRK7', 'DVL2', 'ERBB3', 'HIST1H1A', 'EIF4E', 'MDN1', 'NTRK2', 'NTRK3', 'SMARCC2', 'KDM2A', 'SGTB', 'KATNA1', 'NGF', 'MTOR', 'KLHL42', 'TBC1D5', 'PEAK1', 'MET', 'CRK', 'ERCC6', 'BET1', 'ZMYND8', 'FLT1', 'SMARCD3', 'SRC', 'DCTN4', 'RB1CC1', 'AMBRA1', 'SPRED1', 'POLR2A', 'BAHD1', 'TRA2B', 'UBQLN1', 'NCK1', 'NCK2']
enrichr_library = 'BioPlanet_2019'
genes = [x.strip() for x in gene_list_input]

In [4]:
# Error handling
class NoResults(Exception):
    pass 
class APIFailure(Exception):
    pass

In [5]:
# Enrichr API Function for Manhattan Plot and Bar Chart
# Takes a gene list and Enrichr libraries as input
def Enrichr_API(enrichr_gene_list, all_libraries):

    all_terms = []
    all_pvalues =[] 
    all_adjusted_pvalues = []

    for library_name in all_libraries : 
        ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList'
        genes_str = '\n'.join(enrichr_gene_list)
        description = ''
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }

        response = requests.post(ENRICHR_URL, files=payload)
        if not response.ok:
            raise APIFailure

        data = json.loads(response.text)
        time.sleep(0.5)
        ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
        query_string = '?userListId=%s&backgroundType=%s'
        user_list_id = data['userListId']
        short_id = data["shortId"]
        gene_set_library = library_name
        response = requests.get(
            ENRICHR_URL + query_string % (user_list_id, gene_set_library)
         )
        if not response.ok:
            raise APIFailure

        data = json.loads(response.text)

        if len(data[library_name]) == 0:
            raise NoResults

        short_results_df  = pd.DataFrame(data[library_name][0:10])
        all_terms.append(list(short_results_df[1]))
        all_pvalues.append(list(short_results_df[2]))
        all_adjusted_pvalues.append(list(short_results_df[6]))
        
        results_df  = pd.DataFrame(data[library_name])
        # adds library name to the data frame so the libraries can be distinguished
        results_df['library'] = library_name.replace('_', '')

    return [results_df, short_results_df, all_terms, all_pvalues, all_adjusted_pvalues, str(short_id)]

In [6]:
# Scatter Plot Parameters
significance_value = 0.05

# Bar Chart Parameters
figure_file_format = ['png', 'svg']
output_file_name = 'Enrichr_results_bar'
color = 'lightskyblue'
final_output_file_names = ['{0}.{1}'.format(output_file_name, file_type) for file_type in figure_file_format]

# Hexagonal Canvas Parameters
canvas_color = 'Blue'
num_hex_colored = 10

# Manhattan Plot Parameters
manhattan_colors = ['#003f5c', '#7a5195', '#ef5675', '#ffa600']

In [7]:
def get_library(lib_name):
    '''
    Returns a dictionary mapping each term from the input library to 
    its associated geneset. 
    '''
    raw_lib_data = []

    with urllib.request.urlopen('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=' + lib_name) as f:
        for line in f.readlines():
            raw_lib_data.append(line.decode("utf-8").split("\t\t"))

    name = []
    gene_list = []
    lib_data = {}

    for i in range(len(raw_lib_data)):
        name += [raw_lib_data[i][0]]
        raw_genes = raw_lib_data[i][1].replace('\t', ' ')
        gene_list += [raw_genes[:-1]]
    
    lib_data = {a[0]:a[1].split(' ') for a in zip(name, gene_list)}
    return lib_data


def volcano_plot(library_name, lib):
    '''
    Make volcano plot of odds ratio vs. significance for input library.
    '''
    enrich_results = enrich_crisp(genes, lib, 21000, True)

    res_df = pd.DataFrame(
        [ [
            term, 
            res.pvalue, 
            res.odds_ratio
        ] for (term, res) in enrich_results ], 
        columns=['term', 'pvalue', 'odds_ratio']
    )

    res_df['log_pval'] = np.negative(np.log10(res_df['pvalue']))

    return res_df

In [8]:
lib_data = get_library(enrichr_library)
plot = volcano_plot(enrichr_library, lib_data)
plot.to_csv('volcano_plot_data.csv', index=False)